# 🔧 ОПТИМИЗАЦИЯ ПАРАМЕТРОВ MARKET ZONE CLASSIFIER

## 🎯 ЦЕЛЬ
Найти оптимальные параметры классификатора зон для максимального качества классификации

## 📊 МЕТОДЫ ОПТИМИЗАЦИИ
1. **Bayesian Optimization** (рекомендуется) - самый эффективный
2. **Genetic Algorithm** - хорошо работает, знакомый метод
3. **Random Search** - для быстрого старта

## 🎯 МЕТРИКИ КАЧЕСТВА
- **Zone Stability (30%)** - стабильность зон
- **Zone Separation (30%)** - разделение по доходности
- **Economic Value (40%)** - экономическая ценность


## 📚 ИМПОРТ БИБЛИОТЕК


In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import os
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Добавляем пути
sys.path.append('.')

print("✅ Библиотеки импортированы")


✅ Библиотеки импортированы


## 🔧 ИМПОРТ МОДУЛЯ ОПТИМИЗАТОРА


In [ ]:
!pip install scikit-optimize

In [1]:
# Импортируем модуль оптимизатора
import zone_optimizer
import importlib

# Перезагружаем модуль, чтобы получить последние изменения
importlib.reload(zone_optimizer)

# Импортируем функции и классы
from zone_optimizer import ZoneClassifierOptimizer, load_btc_data

print("✅ Модуль оптимизатора импортирован и перезагружен")
print(f"   Функция load_btc_data принимает base_path: {'base_path' in load_btc_data.__code__.co_varnames}")


✅ Модуль оптимизатора импортирован и перезагружен
   Функция load_btc_data принимает base_path: True


## 📊 ЗАГРУЗКА ДАННЫХ


In [2]:
print("📊 ЗАГРУЗКА ДАННЫХ")
print("=" * 50)

# 🎯 УКАЖИТЕ ПУТЬ К КОРНЮ ПРОЕКТА (где лежат файлы df_btc_*.csv)
# Если оставить None, путь определится автоматически
import os

DATA_BASE_PATH = None  # Можно явно указать: r"C:\Users\XE\Desktop\CRYPTO_2025"

if DATA_BASE_PATH is None:
    # Определяем путь автоматически
    current_dir = os.getcwd()
    # Если мы в подпапке market_zone_classifier, поднимаемся на уровень выше
    if os.path.basename(current_dir) == 'market_zone_classifier':
        DATA_BASE_PATH = os.path.dirname(current_dir)
    else:
        DATA_BASE_PATH = current_dir

print(f"📁 Базовый путь к данным: {DATA_BASE_PATH}")

# Загружаем данные BTC
btc_data = load_btc_data(base_path=DATA_BASE_PATH)

# Выбираем таймфрейм для оптимизации
timeframe = '1h'  # Можно изменить на '15m', '4h', '1d' и т.д.

if btc_data and timeframe in btc_data:
    df = btc_data[timeframe]
    print(f"\n✅ Загружены данные для таймфрейма {timeframe}")
    print(f"   Записей: {len(df)}")
    print(f"   Период: {df.index[0]} - {df.index[-1]}")
    print(f"   Колонки: {list(df.columns)}")
elif btc_data:
    # Берем первый доступный
    timeframe = list(btc_data.keys())[0]
    df = btc_data[timeframe]
    print(f"\n✅ Используем доступный таймфрейм {timeframe}")
    print(f"   Записей: {len(df)}")
    print(f"   Период: {df.index[0]} - {df.index[-1]}")
    print(f"   Колонки: {list(df.columns)}")
else:
    raise ValueError("❌ Не удалось загрузить данные! Убедитесь, что файлы находятся в корне проекта.")

print(f"\n📈 Статистика данных:")
print(f"   Средняя цена: ${df['close'].mean():.2f}")
print(f"   Волатильность: {df['close'].std():.2f}")


📊 ЗАГРУЗКА ДАННЫХ
📁 Базовый путь к данным: c:\Users\XE\Desktop\CRYPTO_2025
📁 Базовый путь: c:\Users\XE\Desktop\CRYPTO_2025

📊 ЗАГРУЗКА ДАННЫХ BTC ИЗ ФАЙЛОВ
✅ 15m: Загружено из df_btc_15m_complete.csv (61000 записей)
✅ 30m: Загружено из df_btc_30m_complete.csv (30000 записей)
✅ 1h: Загружено из df_btc_1h_complete.csv (15000 записей)
✅ 4h: Загружено из df_btc_4h_complete.csv (3000 записей)
✅ 1d: Загружено из df_btc_1d.csv (401 записей)
⚠️ 1d: Volume отсутствует, добавляем синтетический

✅ Загружены данные для таймфрейма 1h
   Записей: 15000
   Период: 2024-02-10 22:00:00 - 2025-10-27 21:00:00
   Колонки: ['open', 'high', 'low', 'close', 'volume']

📈 Статистика данных:
   Средняя цена: $85337.86
   Волатильность: 21251.18


## 🎯 НАСТРОЙКА ЦЕЛЕЙ ОПТИМИЗАЦИИ (МЕТРИК) И РЕЖИМА


In [3]:
print("🎯 НАСТРОЙКА ЦЕЛЕЙ ОПТИМИЗАЦИИ")
print("=" * 50)
print("\n📊 ВЕСА МЕТРИК (что важно для оптимизации):")
print("\nТекущие веса:")
print("  - Zone Stability: 30% - стабильность зон")
print("  - Zone Separation: 30% - разделение зон по доходности")
print("  - Economic Value: 40% - экономическая ценность")
print("\n💡 МОЖНО ИЗМЕНИТЬ веса в зависимости от целей:")
print("  - Больше стабильности: stability=0.5, separation=0.2, economic=0.3")
print("  - Больше разделения: stability=0.2, separation=0.5, economic=0.3")
print("  - Больше экономики: stability=0.2, separation=0.2, economic=0.6")

# Настройка весов метрик
metric_weights = {
    'stability': 0.3,    # Стабильность зон (30%)
    'separation': 0.3,   # Разделение зон (30%)
    'economic': 0.4      # Экономическая ценность (40%)
}

# Выберите метод оптимизации:
# 'bayesian' - Bayesian Optimization (РЕКОМЕНДУЕТСЯ)
# 'genetic' - Genetic Algorithm
# 'nsga2'   - Multi-Objective NSGA-II (Pareto фронт)
# 'random'  - Random Search

optimization_method = 'nsga2'

print(f"\n✅ Выбранный метод: {optimization_method}")
print(f"✅ Веса метрик (используются только для комбинированных режимов): {metric_weights}")

# Создаем оптимизатор с настройками метрик
optimizer = ZoneClassifierOptimizer(
    df, 
    optimization_method=optimization_method,
    metric_weights=metric_weights
)

print("✅ Оптимизатор создан с настройками метрик")


🎯 НАСТРОЙКА ЦЕЛЕЙ ОПТИМИЗАЦИИ

📊 ВЕСА МЕТРИК (что важно для оптимизации):

Текущие веса:
  - Zone Stability: 30% - стабильность зон
  - Zone Separation: 30% - разделение зон по доходности
  - Economic Value: 40% - экономическая ценность

💡 МОЖНО ИЗМЕНИТЬ веса в зависимости от целей:
  - Больше стабильности: stability=0.5, separation=0.2, economic=0.3
  - Больше разделения: stability=0.2, separation=0.5, economic=0.3
  - Больше экономики: stability=0.2, separation=0.2, economic=0.6

✅ Выбранный метод: nsga2
✅ Веса метрик (используются только для комбинированных режимов): {'stability': 0.3, 'separation': 0.3, 'economic': 0.4}
✅ Оптимизатор создан с настройками метрик


## 🧬 ЗАПУСК ОПТИМИЗАЦИИ


In [4]:
# Перезагружаем модуль, чтобы получить последние исправления
import importlib
import sys

# Удаляем модуль из кэша, если он есть
if 'zone_optimizer' in sys.modules:
    del sys.modules['zone_optimizer']

# Импортируем заново
import zone_optimizer
importlib.reload(zone_optimizer)

# Пересоздаем оптимизатор с обновленным модулем
from zone_optimizer import ZoneClassifierOptimizer
optimizer = ZoneClassifierOptimizer(
    df, 
    optimization_method=optimization_method,
    metric_weights=metric_weights
)

print("✅ Модуль перезагружен и оптимизатор создан")

print("🚀 ЗАПУСК ОПТИМИЗАЦИИ")
print("=" * 50)
print(f"⏰ Это может занять несколько минут...")
print()

# Запускаем оптимизацию
# Для Bayesian: n_calls=100 (количество итераций)
# Для Genetic: population_size=30, generations=50
# Для Random: n_trials=500

if optimization_method == 'bayesian':
    results = optimizer.optimize(n_calls=80, n_initial_points=20)
elif optimization_method == 'genetic':
    results = optimizer.optimize(population_size=40, generations=60)
elif optimization_method == 'nsga2':
    results = optimizer.optimize(population_size=60, generations=80)
else:
    results = optimizer.optimize(n_trials=800)

print("\n🎉 ОПТИМИЗАЦИЯ ЗАВЕРШЕНА!")


✅ Модуль перезагружен и оптимизатор создан
🚀 ЗАПУСК ОПТИМИЗАЦИИ
⏰ Это может занять несколько минут...

🚀 Запуск оптимизации методом: nsga2



🎉 ОПТИМИЗАЦИЯ ЗАВЕРШЕНА!


## 📊 РЕЗУЛЬТАТЫ ОПТИМИЗАЦИИ


In [5]:
print("🏆 РЕЗУЛЬТАТЫ ОПТИМИЗАЦИИ")
print("=" * 50)

print(f"\n📊 ИНФОРМАЦИЯ ОБ ОПТИМИЗАЦИИ:")
print(f"   Метод: {results['method']}")

if results['method'] == 'NSGA-II':
    pareto = results['pareto']
    print(f"   Размер Pareto-фронта: {len(pareto)} решений")
    top = sorted(pareto, key=lambda x: x['stability'] + x['separation'] + x['economic'], reverse=True)[:5]
    print("\n🔧 ТОП-5 РЕШЕНИЙ (сумма метрик):")
    for i, sol in enumerate(top, 1):
        print(f" {i}) lb={sol['lookbackPeriod']}, tt={sol['trendThreshold']:.2f}, vp={sol['volatilityPeriod']}, vt={sol['volatilityThreshold']:.2f} | st={sol['stability']:.3f}, sep={sol['separation']:.4f}, eco={sol['economic']:.4f}")
    best = results['best']
    print("\n✅ Выбранное решение (best by sum):")
    print(f"   lb={best['lookbackPeriod']}, tt={best['trendThreshold']:.2f}, vp={best['volatilityPeriod']}, vt={best['volatilityThreshold']:.2f}")
    zones = optimizer.classify_zones(best['lookbackPeriod'], best['trendThreshold'], best['volatilityPeriod'], best['volatilityThreshold'])
else:
    print(f"   Итераций/Попыток: {results['iterations']}")
    print(f"   Лучший Score: {results['score']:.6f}")
    print(f"\n🔧 ОПТИМАЛЬНЫЕ ПАРАМЕТРЫ:")
    print(f"   Lookback Period: {results['lookbackPeriod']}")
    print(f"   Trend Threshold: {results['trendThreshold']:.2f}")
    print(f"   Volatility Period: {results['volatilityPeriod']}")
    print(f"   Volatility Threshold: {results['volatilityThreshold']:.2f}")
    zones = optimizer.classify_zones(results['lookbackPeriod'], results['trendThreshold'], results['volatilityPeriod'], results['volatilityThreshold'])

# Вычисляем доходность
returns = df['close'].pct_change().dropna()

# Выравниваем индексы
common_idx = zones.index.intersection(returns.index)
zones_aligned = zones.loc[common_idx]
returns_aligned = returns.loc[common_idx]

# Получаем детальную разбивку метрик
metrics = optimizer.get_metric_breakdown(zones_aligned, returns_aligned)

print(f"\n📊 ДЕТАЛЬНАЯ РАЗБИВКА МЕТРИК:")
print(f"   Zone Stability: {metrics['stability']:.4f} (вес: {metric_weights['stability']*100:.0f}%)")
print(f"      → Взвешенный вклад: {metrics['stability_weighted']:.6f}")
print(f"   Zone Separation: {metrics['separation']:.6f} (вес: {metric_weights['separation']*100:.0f}%)")
print(f"      → Взвешенный вклад: {metrics['separation_weighted']:.6f}")
print(f"   Economic Value: {metrics['economic']:.6f} (вес: {metric_weights['economic']*100:.0f}%)")
print(f"      → Взвешенный вклад: {metrics['economic_weighted']:.6f}")

if results['method'] != 'NSGA-II':
    print(f"\n🏆 ОБЩИЙ SCORE: {metrics['total_score']:.6f}")

# Статистика по зонам
print(f"\n📊 СТАТИСТИКА ПО ЗОНАМ:")
zone_stats = zones_aligned.value_counts(normalize=True) * 100
for zone, percentage in zone_stats.items():
    zone_name = {
        3: "Strong Bull + High Vol",
        2: "Bull + High Vol",
        1: "Bull + Normal Vol",
        0: "Sideways",
        -1: "Bear + Normal Vol",
        -2: "Bear + High Vol",
        -3: "Strong Bear + High Vol"
    }.get(zone, f"Zone {zone}")
    print(f"   {zone_name}: {percentage:.1f}%")

# Сохраняем результаты best
if results['method'] == 'NSGA-II':
    best_params = {
        'lookbackPeriod': results['best']['lookbackPeriod'],
        'trendThreshold': results['best']['trendThreshold'],
        'volatilityPeriod': results['best']['volatilityPeriod'],
        'volatilityThreshold': results['best']['volatilityThreshold']
    }
else:
    best_params = {
        'lookbackPeriod': results['lookbackPeriod'],
        'trendThreshold': results['trendThreshold'],
        'volatilityPeriod': results['volatilityPeriod'],
        'volatilityThreshold': results['volatilityThreshold']
    }

print(f"\n✅ Параметры готовы к использованию!")


🏆 РЕЗУЛЬТАТЫ ОПТИМИЗАЦИИ

📊 ИНФОРМАЦИЯ ОБ ОПТИМИЗАЦИИ:
   Метод: NSGA-II
   Размер Pareto-фронта: 60 решений

🔧 ТОП-5 РЕШЕНИЙ (сумма метрик):
 1) lb=41, tt=0.81, vp=26, vt=1.54 | st=0.685, sep=0.0000, eco=0.0000
 2) lb=35, tt=0.80, vp=26, vt=1.54 | st=0.685, sep=0.0000, eco=0.0000
 3) lb=44, tt=0.78, vp=26, vt=1.54 | st=0.685, sep=0.0000, eco=0.0000
 4) lb=41, tt=0.80, vp=26, vt=1.54 | st=0.685, sep=0.0000, eco=0.0000
 5) lb=49, tt=0.78, vp=26, vt=1.54 | st=0.685, sep=0.0000, eco=0.0000

✅ Выбранное решение (best by sum):
   lb=41, tt=0.81, vp=26, vt=1.54

📊 ДЕТАЛЬНАЯ РАЗБИВКА МЕТРИК:
   Zone Stability: 0.7609 (вес: 30%)
      → Взвешенный вклад: 0.228270
   Zone Separation: 0.000940 (вес: 30%)
      → Взвешенный вклад: 0.000282
   Economic Value: 0.000285 (вес: 40%)
      → Взвешенный вклад: 0.000114

📊 СТАТИСТИКА ПО ЗОНАМ:
   Sideways: 35.3%
   Bull + Normal Vol: 32.8%
   Bear + Normal Vol: 26.8%
   Bear + High Vol: 1.7%
   Strong Bull + High Vol: 1.4%
   Bull + High Vol: 1.1%
   Str

## 📤 ЭКСПОРТ В PINE SCRIPT


In [ ]:
print("📤 ЭКСПОРТ ОПТИМАЛЬНЫХ ПАРАМЕТРОВ В PINE SCRIPT")
print("=" * 50)

from datetime import datetime
current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

# Выбираем источник параметров (NSGA-II или одиночный метод)
if results['method'] == 'NSGA-II':
    params = results['best']
    score_str = f"st={params['stability']:.3f}; sep={params['separation']:.4f}; eco={params['economic']:.4f}"
else:
    params = results
    score_str = f"score={results['score']:.6f}"

pine_code = f"""
// Оптимизированные параметры Market Zone Classifier
// Таймфрейм: {timeframe}
// Метод оптимизации: {results['method']}
// {score_str}
// Дата оптимизации: {current_time}

// Замените в price_action_classifier.pine строки:
// Manual parameters (used when timeframeMode = "Manual")
lookbackPeriod = input.int({params['lookbackPeriod']}, "Lookback Period", minval=5, maxval=100, group="Manual Settings")
trendThreshold = input.float({params['trendThreshold']:.2f}, "Trend Threshold", minval=0.1, maxval=2.0, step=0.1, group="Manual Settings")
volatilityPeriod = input.int({params['volatilityPeriod']}, "Volatility Period", minval=5, maxval=50, group="Manual Settings")
volatilityThreshold = input.float({params['volatilityThreshold']:.2f}, "Volatility Threshold", minval=0.5, maxval=3.0, step=0.1, group="Manual Settings")
"""

print(pine_code)

# Сохраняем в файл
with open(f'optimized_params_{timeframe}_{datetime.now().strftime("%Y%m%d")}.pine', 'w', encoding='utf-8') as f:
    f.write(pine_code)

print(f"\n✅ Параметры сохранены в файл!")
